In [1]:
import cv2
import numpy as np
import pickle

In [2]:


# Load mô hình phát hiện khuôn mặt
net = cv2.dnn.readNetFromCaffe(
    'deploy.prototxt',
    'res10_300x300_ssd_iter_140000_fp16.caffemodel'
)

# Load mô hình trích xuất đặc trưng
descriptorModel = cv2.dnn.readNetFromTorch('openface.nn4.small2.v1.t7')

# Load mô hình nhận diện đã huấn luyện (VotingClassifier)
model = pickle.load(open('/Users/ttdat/Documents/Do_An_Co_So/ml_face_person_identity.pkl', 'rb'))

# Mở webcam (0 hoặc 1 tùy thiết bị)
cap = cv2.VideoCapture(1)

while True: 
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104, 177, 123), swapRB=False)
    net.setInput(blob)
    detections = net.forward()

    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype(int)

            # Cắt khuôn mặt
            face = frame[startY:endY, startX:endX]
            if face.shape[0] == 0 or face.shape[1] == 0:
                continue

            # Trích xuất đặc trưng 128-D
            faceBlob = cv2.dnn.blobFromImage(face, 1/255.0, (96, 96), (0, 0, 0), swapRB=True, crop=True)
            descriptorModel.setInput(faceBlob)
            vec = descriptorModel.forward()

            # Dự đoán tên người
            name = model.predict(vec)[0]
            try:
                score = model.predict_proba(vec).max()
                label = f"{name}: {score*100:.1f}%"
            except:
                label = name  # Trường hợp dùng voting='hard'

            # Hiển thị kết quả
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    cv2.imshow("Kết quả", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

2025-05-16 22:21:16.638 python[15696:2102231] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


ValueError: X has 128 features, but LogisticRegression is expecting 512 features as input.

: 